# Handling Extra Fields

We'll start looking at the various model configuration options available starting with how Pydantic handles fields present in the data being deserialized that are not defined in the model itself. These are called "extra" fields.

There are a ton of different configuration options, which can be found [here](https://docs.pydantic.dev/latest/api/config)

In [1]:
from pydantic import BaseModel, ConfigDict, ValidationError

In [2]:
class Model(BaseModel):
    field_1: int

In [3]:
m = Model(field_1=10, field_2=20)
m

Model(field_1=10)

As you can see, `field_2` is not present in the representation, and is in fact not present in the model instance dictionary either:

In [4]:
dict(m)

{'field_1': 10}

Pydantic provides us a way to see what fields were provided in the data, but that were not defined in the model:

In [5]:
m.model_extra

As you can see, even that is empty.

So, the default is for Pydantic to totally ignore extra fields present in the data.

We can modify this behavior by configuring our model.

The configuration option we are looking for here, is called `extra`.

Specific documentation for this option, is in the page I linked above, or can be directly accessed [here](https://docs.pydantic.dev/latest/api/config/#pydantic.config.ConfigDict.extra)

From the documentation we see that this `extra` option can be set to one of three possible values:
- `ignore` - the default behavior
- `forbid` - this will cause a validation error is extra fields are encountered in the data
- `allow` - this will actually add the field and it's value to the model, but of course without any validation.

Let's look at how to explicitly set the behavior to `ignore` (which is the default, so we don't really need to do this - just illustrating).

In [6]:
class Model(BaseModel):
    model_config = ConfigDict(extra="ignore")

    field_1: int = 0

In [7]:
Model(field_1=10, extra_1="data")

Model(field_1=10)

Now let's change the behavior to raise a validation exception if extra fields are present.

In [8]:
class Model(BaseModel):
    model_config = ConfigDict(extra="forbid")

    field_1: int = 0

In [9]:
try:
    Model(field_1=10, extra_1="data")
except ValidationError as ex:
    print(ex)

1 validation error for Model
extra_1
  Extra inputs are not permitted [type=extra_forbidden, input_value='data', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/extra_forbidden


As you can see, we now get a validation exception. This is typically used in REST APIs where you want to ensure that request payloads do not contain extra data - it is better to be strict about it and let the caller know there is something wrong with their request payload, as they may not realize they have a problem on their end.

The last option is to actually allow (and handle) extra fields:

In [10]:
class Model(BaseModel):
    model_config = ConfigDict(extra="allow")

    field_1: int = 0

Now we can specify additional (undefined) fields in the data, and Pydantic will add it to the model **instance** (so no common schema here, the **model** schema still stays what it was)

In [11]:
m = Model(field_1=10, extra_1="data")
m

Model(field_1=10, extra_1='data')

As you can see, the ad-hoc field is present in the representation, as well as the instance dictionary:

In [12]:
dict(m)

{'field_1': 10, 'extra_1': 'data'}

The model fields have not changed :

In [13]:
m.model_fields

{'field_1': FieldInfo(annotation=int, required=False, default=0)}

When we serialize the instance, that data will be present:

In [14]:
m.model_dump()

{'field_1': 10, 'extra_1': 'data'}

We can perform some inspection like we covered in an earlier video to see what's going on.

In [15]:
m.model_fields_set

{'extra_1', 'field_1'}

As you can see, the field name `extra_1` is present in this view of the data - that's becuase it **has** been set on the model, and was set from the data, not from a default.

We also have a property to list just those extra fields:

In [16]:
m.model_extra

{'extra_1': 'data'}

And here we get a dictionary contain those extra fields - can prove useful in some cases.

For example, you may query an API that returns a rather complex JSON object. Out of that object, you are only interested in a few pieces of information, but you need to retain the other information for some other purpose (maybe logging). Instead of defining the entire JSON schema with a Pydantic model, you would only define the pieces you need in your Pydantic model, set the model to `allow` and now you have the option of validating only the portions you are interested in, and just taking in the rest of the data as-is. This can also reduce your validation burden - don't validate what you are not going to use.

One last piece of information regarding model configs, we can always recover the specific configuration for a model (or an instance of it) if we need to (often for debugging purposes):

In [17]:
Model.model_config

{'extra': 'allow'}

In [18]:
m.model_config

{'extra': 'allow'}

As you can see from the above, the `model_config` attribute is a simply `dict` object, not a `ConfigDict` object.

You can certainly use a plain dictionary to configure your model:

In [19]:
class Model(BaseModel):
    model_config = {"extra": "allow"}

    field_1: int = 0

In [20]:
Model(extra_1=100)

Model(field_1=0, extra_1=100)

The reason you should probably not do that is because `ConfigDict` is a `typedict`, and so offers some protections against messing up your configuration. And type checkers (and your IDE) should pick up on that before you even run your code. Like providing a string value for a config that actually expects a boolean for example.